In [16]:
#Loading neccesary packages
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [17]:
df_data = pd.read_parquet(r'C:\Users\Hendelle.DESKTOP-V5JNA20\OneDrive\Desktop\Market Basket Analysis\ojt_data.gzip')

In [18]:
# Loading Data

#df_data = pd.read_parquet('ojt_data.gzip')
df_data = df_data[df_data['Quantity Sold']>0].reset_index(drop=True) # remove negative transaction
df_data = df_data[df_data['Quantity Unit of Measure']!='AU'].reset_index(drop=True) # remove UOM AU
df_data = df_data[df_data['Quantity Unit of Measure']!='BD'].reset_index(drop=True) # remove UOM BD
df_data = df_data[df_data['Quantity Unit of Measure']!='UNDEFINED'].reset_index(drop=True) # remove UOM UNDEFINED
df_data.head()

,Transaction Year,Transaction Month,Transaction Day of Month,Transaction Quarter,Transaction Day of Week,Transaction Number,Item Category,Item SubCategory,Item Class,Item SubClass,Item Description,Quantity Sold,Quantity Unit of Measure,Sales (srp*qty)
0,2019,1,7,Q1,Mon,00000000000000062203,Appliances,Major Appliance,Airconditioning,Split Type,EVEREST @ ET-R220ST-HF SPLIT TYPE AC 2HP,1.0,PC,31990.0
1,2019,1,7,Q1,Mon,00000000000000047004,Appliances,Major Appliance,Built In Hob,Gas Burners,LA GERMANIA GH820X HOB 2G 45CM,1.0,PC,12650.0
2,2019,1,7,Q1,Mon,00000000000000062236,Appliances,Major Appliance,Laundry,Single Tub,SHARP ES-W600(BL) ST WM 6KG,1.0,PC,3998.0
3,2019,1,7,Q1,Mon,00000000000000062158,Appliances,Major Appliance,Laundry,Spin Dryer,WWH S65XW SPIN DRY 6.5KG,1.0,PC,3695.0
4,2019,1,7,Q1,Mon,00000000000000062227,Appliances,Major Appliance,Laundry,Topload,LG T2309VSAM TL WM 9KG INV,1.0,PC,24995.0


# Data Preparation

### Data Cleaning

In [19]:
#Checking of missing values
missing_values_count = df_data.isnull().sum()
missing_values_count[0:25]

Transaction Year            0
Transaction Month           0
Transaction Day of Month    0
Transaction Quarter         0
Transaction Day of Week     0
Transaction Number          0
Item Category               0
Item SubCategory            0
Item Class                  0
Item SubClass               0
Item Description            0
Quantity Sold               0
Quantity Unit of Measure    0
Sales (srp*qty)             0
dtype: int64

In [20]:
df_data.dropna(axis='index', how='any')

#removes spaces from beginning and end
df_data['Item Category'] = df_data['Item Category'].str.strip() 
df_data['Item SubCategory'] = df_data['Item SubCategory'].str.strip()
df_data['Item Class'] = df_data['Item Class'].str.strip()
df_data['Item SubClass'] = df_data['Item SubClass'].str.strip()
df_data['Item Description'] = df_data['Item Description'].str.strip()

# Adding new column: Unique ID
df_data['Unique ID']=df_data['Transaction Year'].astype(str) + df_data['Transaction Month'].astype(str)+ df_data['Transaction Day of Month'].astype(str)+ df_data['Transaction Number'].astype(str)

#converting invoice number to be string
df_data['Unique ID'] = df_data['Unique ID'].astype('str')

df_data.reset_index(drop=True,inplace=True)
df_data.head()

,Transaction Year,Transaction Month,Transaction Day of Month,Transaction Quarter,Transaction Day of Week,Transaction Number,Item Category,Item SubCategory,Item Class,Item SubClass,Item Description,Quantity Sold,Quantity Unit of Measure,Sales (srp*qty),Unique ID
0,2019,1,7,Q1,Mon,00000000000000062203,Appliances,Major Appliance,Airconditioning,Split Type,EVEREST @ ET-R220ST-HF SPLIT TYPE AC 2HP,1.0,PC,31990.0,20191700000000000000062203
1,2019,1,7,Q1,Mon,00000000000000047004,Appliances,Major Appliance,Built In Hob,Gas Burners,LA GERMANIA GH820X HOB 2G 45CM,1.0,PC,12650.0,20191700000000000000047004
2,2019,1,7,Q1,Mon,00000000000000062236,Appliances,Major Appliance,Laundry,Single Tub,SHARP ES-W600(BL) ST WM 6KG,1.0,PC,3998.0,20191700000000000000062236
3,2019,1,7,Q1,Mon,00000000000000062158,Appliances,Major Appliance,Laundry,Spin Dryer,WWH S65XW SPIN DRY 6.5KG,1.0,PC,3695.0,20191700000000000000062158
4,2019,1,7,Q1,Mon,00000000000000062227,Appliances,Major Appliance,Laundry,Topload,LG T2309VSAM TL WM 9KG INV,1.0,PC,24995.0,20191700000000000000062227


In [21]:
#separating transactions

basket = df_data.groupby(['Unique ID', 'Item SubClass'])['Quantity Sold'].sum().unstack()
basket

C:\Users\Hendelle.DESKTOP-V5JNA20\anaconda3\lib\site-packages\pandas\core\reshape\reshape.py:125: RuntimeWarning: overflow encountered in long_scalars
  num_cells = num_rows * num_columns


IndexError: index 213736423 is out of bounds for axis 0 with size 213697656

In [ ]:
#converting all positive values to 1 and everything else to 0

def my_encode_units(x):
    if x >= 0: return 1
    elif x < 0: return 0
    else: return 0
    
basket_set = basket.applymap(my_encode_units)
basket_set

# Training Model

In [ ]:
#Generating frequent itemsets

freq_items = apriori(basket_set, min_support=0.01, use_colnames=True, max_len=3)
freq_items.sort_values(by = "support", ascending = False)
#freq_items[freq_items['itemsets'].str.contains('Fragrance Oil',regex = False)]

In [ ]:
#Generating rules
rules = association_rules(freq_items, metric='confidence',min_threshold=0.05)
rules.sort_values(by = "support", ascending = False)
#rules[rules['consequents'].str.contains('Fragrance Oil', regex=False)]


In [ ]:
#Filtering rules based on condition

filteredrules = rules[(rules['lift'] >=2) & (rules['confidence'] >= 0.3)]
filteredrules